In [ ]:
!pip install transformers

In [ ]:
import pandas as pd

df = pd.read_csv("/content/punjabiData (1).csv")

In [ ]:
df.head()

In [ ]:
df["sentiment"].value_counts()

,count
sentiment,
Positive,2166
Negative,2166
Neutral,2166


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.preprocessing import LabelEncoder

X = df["sentence"]
y = df["sentiment"]

le = LabelEncoder()
labels = le.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size = 0.2, random_state=42)

In [ ]:
print(set(labels))

{np.int64(0), np.int64(1), np.int64(2)}


In [ ]:
import numpy as np
num_classes = len(np.unique(labels))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
from transformers import TFBertForSequenceClassification, BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
def tokenize_data(texts, labels):
    tokenized = tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=128,  # Reduced from 512 for efficiency with Punjabi
        return_tensors='tf'
    )
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask']
    }, tf.one_hot(labels, depth=num_classes)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(tokenize_data(X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices(tokenize_data(X_test, y_test))

In [ ]:
batch_size = 64
train_dataset = train_dataset.shuffle(len(X_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels = num_classes
)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.compile(
    loss = keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=2e-5),
    metrics = ["accuracy"]
)

In [ ]:
model.fit(train_dataset, epochs = 5, validation_data=test_dataset)

Epoch 1/5
82/82 [==============================] - 187s 2s/step - loss: 0.8056 - accuracy: 0.6385 - val_loss: 0.6208 - val_accuracy: 0.7569
Epoch 2/5
82/82 [==============================] - 142s 2s/step - loss: 0.4592 - accuracy: 0.8270 - val_loss: 0.4133 - val_accuracy: 0.8446
Epoch 3/5
82/82 [==============================] - 131s 2s/step - loss: 0.2873 - accuracy: 0.9007 - val_loss: 0.3507 - val_accuracy: 0.8685
Epoch 4/5
82/82 [==============================] - 141s 2s/step - loss: 0.2204 - accuracy: 0.9267 - val_loss: 0.3557 - val_accuracy: 0.8869
Epoch 5/5
82/82 [==============================] - 141s 2s/step - loss: 0.1638 - accuracy: 0.9444 - val_loss: 0.3642 - val_accuracy: 0.8808


In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"\nTest Accuracy: {test_acc:.4f}")

21/21 [==============================] - 11s 507ms/step - loss: 0.3642 - accuracy: 0.8808

Test Accuracy: 0.8808


In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='tf'
    )
    outputs = model(inputs)
    logits = outputs.logits
    probabilities = tf.nn.softmax(logits, axis=-1)
    predicted_class = tf.argmax(probabilities, axis=-1).numpy()[0]
    return le.inverse_transform([predicted_class])[0], probabilities.numpy()[0]

In [ ]:
sample_text = "ਮੈਂ ਇਸ ਨਾਲ ਬਹੁਤ ਖੁਸ਼ ਹਾਂ"
sentiment, confidence = predict_sentiment(sample_text)
print(f"\nText: {sample_text}")
print(f"Predicted Sentiment: {sentiment}")
print(f"Confidence: {confidence}")


Text: ਮੈਂ ਇਸ ਨਾਲ ਬਹੁਤ ਖੁਸ਼ ਹਾਂ
Predicted Sentiment: Positive
Confidence: [0.00307779 0.00518373 0.9917385 ]
